<h3 align="center">DS 211, Nov 2019 &emsp; &emsp; &emsp; &emsp;&emsp;&emsp;&emsp;Numerical Optimization
Mini-Project 2&emsp;&emsp;&emsp;&emsp;&emsp; &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
CDS, IISc</h3>
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

| No. | Member | Contribution |
| --- | --- | --- |
| 1 | Raghav | 1a,b,c |
| 3 | Rohit | 2d, 3d, 3e |
| 2 | Akanksha | 2a,b,c |
| 4 | Kaushik | 3a,b,c |
| 5 | Prahsant | 4a,b,c |

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
<h1 align="center">Question 1</h1>

## (a) 
Data is generated using following genrating process.
$$ y = exp(-rt) +v$$
where $v$ is N(0,2) and 0$\leq$ t$\leq$ 5. Saved this generated data as x_values and y_values. After this I will be using this data to solve for value of $r$.Following is the code for Generating data and saving in a ".csv" format. 


In [ ]:
import numpy as np
import pandas as pd 
#generating t values
tvalues , steps= np.linspace(0, 5 , 500, 
                             endpoint = True,
                             retstep = True)
r = 2.7 #given in question
y = np.zeros((500),np.float64)
for i in range(0, 500):
    # generating a random number from normal distribution with mean 0 and var = 2
    x  = np.random.normal(loc= 0, scale = 2.0,size = 1)
    y[i] = np.exp(- r* tvalues[i]) +x
        
Data = np.asarray([(tvalues[i], y[i]) for i in range(0,500)])
Data_df  = pd.DataFrame(Data).rename(columns= {0: "X", 1:"Y"})
#Saving data to CSV file
Data_df.to_csv("./Data.csv", index = False)

## (b)
Modeling data as $y = exp(-rt) +v$ and findind value of $r$ which best fits the data.For this we can model a least square problem as below. 
\begin{equation*}
 \min_{r} \sum_{i=1}^n (exp(-rt_{i}) - y_{i})^2 
\end{equation*}
Where $t_{i}$ and $y_{i}$ came from x_values and y_values generated from last question. Total number fo points generated was 500. So, value of $n$ is 500 here.Gradient of above loss function with respect to r is - 
\begin{equation*}
 J = \sum_{i=1}^n -2exp(-rt_{i})*(exp(-rt_{i}) - y_{i})t_{i} 
\end{equation*}
We can solve this non-linear least square problem by using Gauss Newten method in which instant for finding double derivatives we approximate it as $J^TJ$. 
So, expression for iterate step $p_{k}$ will become - 
\begin{equation*}
 J^TJ*p_k = -J^Tr 
\end{equation*}
Above expression will become very simple if variable dimension is 1(In case of this question).After that $alpha$ can be found by backtracking to do $x_{k+1} = x_{k} +aplha*p_{k}$.


In [50]:
import pandas as pd
import numpy as np
from time import time
def Loss(r, xvalues, yvalues):
    with np.errstate(all='ignore'):
    
        A= np.sum(np.square(np.exp(-r*xvalues) - yvalues))/xvalues.shape[0]
    return A
    
def Grad_Loss(r, xvalues,yvalues):
    return 2*np.sum(np.multiply((np.exp(-r*xvalues) - yvalues),
                              -np.multiply(np.exp(-r*xvalues),
                                           xvalues)))/xvalues.shape[0]    
def back_track(x, p, xvalues,yvalues):
    alpha = 1.0
    c = 1e-4
    roh = 0.9
    grad = Grad_Loss(x, xvalues, yvalues)
    x2 = x
    while Loss(x2,xvalues, yvalues) > Loss(x,xvalues, yvalues) + c*alpha*np.dot(np.transpose(grad),p):
        alpha = roh*alpha
        x2 = x + alpha*p
    return alpha
Data = pd.read_csv("/home/raghav/Mtech/COURSES/Numerical Optimization/DS211_Mini_Project2/Data.csv")
xvalues = np.asarray(Data["X"])
yvalues =  np.asarray(Data["Y"])
#we have to calculate r 
#initializing r
r_o = 0
epsilon =1e-8
r_1 = 0
itera = 0
start_time = time()
while 1:
    itera = itera+1
    r_o = r_1
    J= Grad_Loss(r_o,xvalues,yvalues)
    L = Loss(r_o,xvalues,yvalues)
    p_k = -L / (1e-10+J)
    alpha = back_track(r_o, p_k,xvalues, yvalues)
    r_1 = r_o +alpha* p_k

    if(np.abs(Grad_Loss(r_o,xvalues,yvalues)) <epsilon):
        print("gtol reached")
        break
    if np.abs(r_o - r_1)<epsilon:
        print("xtol reached")
        break
    if np.abs(Loss(r_o,xvalues,yvalues) - Loss(r_1,xvalues,yvalues))<epsilon:
        print("ftol reached")
        break
      
end_time = time()
print("number of interations", itera)
print("execution time for Implemented Quasi-Newton =", "%.5f"%(end_time- start_time), "seconds")
print("Value of r  = ", r_1,", Loss = " ,Loss(r_1,xvalues,yvalues),
      ",Gradient = " ,Grad_Loss(r_1,xvalues,yvalues))

ftol reached
number of interations 41
execution time for Implemented Quasi-Newton = 0.18302 seconds
Value of r  =  2.4823034201494605 , Loss =  3.4768432301470193 ,Gradient =  -8.200927294469562e-05


# (c)
Used built in function of least squares from scipy library in python. With all tolerance values equal to 1e-8(by all tolerance values we meant tolerance on loss change, variable change and loss gradient), We compared both Gauss Newton and levenberg marquadt methods. Following are the results.


| Algorithm | number of interations | execution time | Value of r |
| --- | --- | --- | ---|
| Gauss Newton | 41 | 0.18302 | 2.4823034 |
| levenberg marquadt | 17 | 0.00080 | 2.4902026|



In [51]:
from scipy import optimize
import pandas as pd
import numpy as np
from time import time

In [52]:
def Func(x, xvalues, yvalues):
    return np.sum(np.square(np.exp(-x*xvalues) - yvalues))/xvalues.shape[0]

def Grad_Loss(x,xvalues, yvalues):
    return 2*np.sum(np.multiply((np.exp(-x*xvalues) - yvalues),
                              -np.multiply(np.exp(-x*xvalues),
                                           xvalues)))/xvalues.shape[0]


In [53]:
Data = pd.read_csv("/home/raghav/Mtech/COURSES/Numerical Optimization/DS211_Mini_Project2/Data.csv")
xvalues = np.asarray(Data["X"])
yvalues =  np.asarray(Data["Y"])
r_o =0
start_time = time()
r_1 = optimize.least_squares(fun= Func, x0= r_o, method = "lm", 
                             loss= "linear", args=(xvalues, yvalues))
end_time = time()

In [58]:
print(r_1.message)
print("number of interations = ", r_1.nfev)
print("execution time for Implemented Quasi-Newton =", "%.5f"%(end_time- start_time), "seconds")
print("Value of r  = ", r_1.x[0]," Loss = " ,r_1.fun[0], ",   Gradient = " ,r_1.grad[0])

`ftol` termination condition is satisfied.
number of interations =  17
execution time for Implemented Quasi-Newton = 0.00080 seconds
Value of r  =  2.490202655127126  Loss =  3.476842892941484 ,   Gradient =  -1.2300623873949684e-05
